In [2]:
import numpy as np

In [33]:
class MF():

    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.

        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """

        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 10 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)


In [49]:
R = np.array([
    [5, 3, 0, 1],
    [4, 0, 0, 1],
    [1, 1, 0, 5],
    [1, 0, 0, 4],
    [0, 1, 5, 4],
])

mf = MF(R, K=100, alpha=0.1, beta=0.01, iterations= 20)

In [50]:
mf.train()

Iteration: 10 ; error = 0.7987
Iteration: 20 ; error = 0.2230


[(0, 5.866776427663238),
 (1, 5.646671979170431),
 (2, 5.413632178591395),
 (3, 4.982855664452385),
 (4, 4.104305992137418),
 (5, 2.6335722853440435),
 (6, 1.446907938465551),
 (7, 1.046652408794197),
 (8, 0.9067930547087689),
 (9, 0.7987279284296389),
 (10, 0.6956349801379261),
 (11, 0.6189771606808363),
 (12, 0.5459748611195608),
 (13, 0.4810392723705818),
 (14, 0.4259650542423384),
 (15, 0.37429201913971155),
 (16, 0.3252697946639212),
 (17, 0.288803742670788),
 (18, 0.25149071139377915),
 (19, 0.22295340029673047)]

In [51]:
print(f'Q shape: {mf.Q.shape}')
print(f'P shape: {mf.P.shape}')

Q shape: (4, 100)
P shape: (5, 100)


In [52]:
mf.full_matrix()

array([[5.00779033, 2.957786  , 4.12733255, 1.02306972],
       [3.97338964, 2.10268984, 3.71829876, 0.999536  ],
       [1.07400382, 0.91385684, 4.76007672, 4.99435061],
       [0.98461685, 0.59750395, 4.15453455, 3.97405281],
       [2.08858155, 1.13731879, 4.88814433, 4.03765378]])

In [53]:
mf.get_rating(1,1)

2.1026898447521587